In [1]:
#連結Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
#查看Google Drive的內容
!ls /content/gdrive/My\ Drive/TA/1082

AI_CUP-C1_P1_DevBaseline.pdf  CNN_VGG16.ipynb		   Model
C1-P1_Dev		      Data_Preprocess.ipynb	   Old_Data
C1-P1_Train		      dev.csv			   TestEnsemble.ipynb
CNN_DenseNet-121.ipynb	      mango_dev_planet_data.npz    train.csv
CNN_ResNeXt-50.ipynb	      mango_train_planet_data.npz


In [3]:
%tensorflow_version 1.x
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import layers
from keras import models
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier
from scipy.stats import mode
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

TensorFlow 1.x selected.


Using TensorFlow backend.


# **Load model & weights**

In [4]:
model_resnext50 = models.load_model('/content/gdrive/My Drive/TA/1082/Model/ResNeXt50/TestCNN4.h5')
model_resnext50.load_weights('/content/gdrive/My Drive/TA/1082/Model/ResNeXt50/TestCNN4_weights.h5')















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [0]:
model_vgg16 = models.load_model('/content/gdrive/My Drive/TA/1082/Model/VGG16/TestCNN2.h5')
model_vgg16.load_weights('/content/gdrive/My Drive/TA/1082/Model/VGG16/TestCNN2.ckpt')

In [6]:
model_densenet121 = models.load_model('/content/gdrive/My Drive/TA/1082/Model/DenseNet121/TestCNN5.h5')
model_densenet121.load_weights('/content/gdrive/My Drive/TA/1082/Model/DenseNet121/TestCNN5_weights.h5')

# **Prepare dev dataset**

In [7]:
#Load dev dataset
data = np.load('/content/gdrive/My Drive/TA/1082/mango_dev_planet_data.npz')
X, Y = data['arr_0'], data['arr_1']
print('Loaded shape: ', X.shape, Y.shape)

Loaded shape:  (800, 224, 224, 3) (800, 3)


In [0]:
#Rescale dev dataset
X/=255.0

# **Pedict & Vote**

In [0]:
#Predict
resnext50_predict = model_resnext50.predict_classes(X)
vgg16_predict = model_vgg16.predict_classes(X)
densenet121_predict = model_densenet121.predict_classes(X)

In [0]:
#Start to vote
afterVoteResult = []

In [42]:
#Vote
for i in range(0, len(resnext50_predict)):
  voteArray = []

  #Voting weight = (1, 3, 1)
  voteArray.append(resnext50_predict[i])
  for j in range(1, 4):
    voteArray.append(vgg16_predict[i])
  voteArray.append(densenet121_predict[i])

  #Convert to array
  voteArray = np.array(voteArray)
  #Call mode() to get most common label in voteArray[i]
  voteResult, _ = mode(voteArray)

  #Save voteResult
  afterVoteResult.append(voteResult[0])

#Convert to array
afterVoteResult = np.array(afterVoteResult)
print(len(afterVoteResult))
print(afterVoteResult)

800
[1 0 1 1 2 1 2 0 0 0 0 1 2 1 1 1 1 0 0 1 1 0 2 0 1 0 2 2 0 1 0 0 1 2 0 0 0
 0 0 0 0 1 0 1 2 2 1 0 2 0 1 1 1 1 1 0 1 1 2 0 0 0 2 0 0 2 1 0 2 1 2 1 1 2
 2 1 0 1 2 0 0 2 1 1 2 2 2 1 1 1 1 1 0 2 1 2 1 0 2 1 1 1 1 2 1 1 0 0 1 2 2
 0 1 0 0 2 2 1 0 0 1 0 2 1 2 1 0 1 2 1 0 0 2 1 0 2 1 1 0 2 1 0 2 0 1 0 1 1
 1 1 1 0 1 1 1 0 1 0 2 1 2 1 1 1 2 2 0 2 0 2 1 1 2 2 2 2 0 1 0 1 2 1 0 2 1
 2 0 1 2 1 2 2 1 1 1 0 0 2 2 0 2 0 1 0 1 0 0 0 2 2 2 1 0 2 1 2 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 0 2 0 2 0 1 1 2 0 1 0 0 2 2 1 1 0 2 2 2 1 1 1 2
 2 2 1 1 0 2 1 1 0 1 1 2 1 1 0 0 2 2 1 1 0 1 0 1 2 1 0 1 2 1 2 2 0 2 1 1 1
 1 1 1 2 2 0 2 0 2 2 1 2 1 2 2 0 2 1 0 2 1 1 1 0 0 1 1 1 2 1 1 1 1 2 1 0 2
 1 1 1 1 2 0 1 1 2 0 1 0 0 0 0 0 1 2 1 1 0 1 0 0 2 1 0 0 1 1 0 2 1 0 1 1 0
 2 0 1 1 0 1 1 2 0 2 0 1 0 1 1 2 1 2 0 2 2 2 0 2 1 0 1 2 0 0 0 1 2 1 0 1 0
 0 2 1 0 1 1 0 1 2 1 2 2 1 0 0 0 2 0 2 1 1 1 1 1 0 0 0 1 2 1 1 0 0 1 0 1 0
 0 1 0 1 1 1 2 2 1 2 0 1 0 1 0 1 2 0 2 1 0 1 1 0 1 0 1 2 2 2 0 1 2 1 0 1 0
 0 0 0 1 1 0 1 2 2 1 

In [36]:
#dev dataset label
Y_cat = Y.argmax(axis=-1)
print(Y_cat)

[1 0 1 1 2 1 2 0 1 0 0 1 2 1 2 1 1 0 0 1 0 0 2 0 1 0 2 2 1 1 1 0 2 2 0 0 0
 0 0 0 1 2 0 2 2 2 1 0 2 0 1 1 1 2 2 0 1 0 2 0 1 1 2 0 1 2 2 0 2 1 2 1 1 2
 2 2 1 0 2 0 0 2 1 1 2 2 2 2 0 1 2 2 1 2 2 2 1 0 2 2 2 1 1 2 1 1 1 1 1 2 2
 0 1 0 0 2 2 0 2 0 1 0 2 1 2 1 0 1 2 1 0 0 0 1 0 2 1 1 0 2 0 1 2 0 0 0 2 1
 1 1 1 1 1 1 1 1 1 0 2 1 2 1 0 1 2 2 1 2 0 2 1 2 2 2 2 1 0 0 0 1 2 2 0 2 1
 2 0 0 2 1 2 2 1 1 2 1 0 2 1 0 2 2 1 0 1 2 0 1 2 2 2 1 1 2 0 2 1 2 1 1 0 1
 1 1 1 1 0 0 1 1 0 2 1 1 0 1 2 0 2 0 1 2 2 0 1 0 2 2 2 1 0 1 2 2 2 1 1 1 2
 2 2 1 1 0 2 1 2 0 1 2 2 1 1 1 1 2 2 0 2 0 1 0 0 1 1 1 1 2 1 2 2 0 2 2 1 1
 0 1 1 2 2 0 2 0 2 2 1 2 1 2 2 0 2 1 0 2 0 1 1 0 0 1 1 2 2 1 1 1 0 2 1 1 2
 0 2 1 1 2 1 1 1 2 0 2 0 0 0 0 0 1 2 1 2 0 1 0 1 2 1 0 0 1 0 0 2 1 1 1 1 1
 2 2 1 2 0 2 1 2 0 2 0 1 0 1 1 2 1 1 0 2 2 2 1 2 1 0 1 1 1 1 0 0 2 1 0 2 1
 1 2 0 0 2 1 0 1 1 1 2 2 1 1 0 0 2 0 2 1 1 1 2 2 0 2 0 1 2 2 1 1 0 1 0 1 0
 0 1 0 1 1 1 2 2 2 2 0 0 0 1 0 2 2 1 2 2 0 1 1 0 1 0 1 2 2 1 0 1 2 1 1 1 0
 0 0 0 1 2 1 1 2 2 1 2 1 

# **Confusion matrix**

In [43]:
#Confusion matrix by pandas
pd.crosstab(Y_cat, afterVoteResult, rownames=['labels'], colnames=['prediction'])

prediction,0,1,2
labels,,,
0,181,58,4
1,56,220,17
2,10,65,189


In [44]:
#Confusion matrix by sklearn
c = confusion_matrix(Y_cat, afterVoteResult)
print(c)

[[181  58   4]
 [ 56 220  17]
 [ 10  65 189]]


In [45]:
#Precison, recall and f1-score
print(classification_report(Y_cat, afterVoteResult, target_names=['A', 'B', 'C']))

              precision    recall  f1-score   support

           A       0.73      0.74      0.74       243
           B       0.64      0.75      0.69       293
           C       0.90      0.72      0.80       264

    accuracy                           0.74       800
   macro avg       0.76      0.74      0.74       800
weighted avg       0.75      0.74      0.74       800

